In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🤖 Proactive Business Analytics System (Capstone Final)\n",
    "## Multi-Agent Production Demo\n",
    "\n",
    "**Powered by Google Gemini 2.5 Flash**\n",
    "\n",
    "This notebook demonstrates the **live execution** of the Capstone-Agents-MVP system. Unlike prototypes using mock data, this system processes real files using autonomous agents equipped with Python tools.\n",
    "\n",
    "### 🚀 Real Capabilities demonstrated:\n",
    "- **Filepath Passing Architecture:** Efficiently handling large datasets.\n",
    "- **Statistical Tools:** Real-time anomaly detection (IQR/Z-Score).\n",
    "- **Gemini 2.5 Integration:** Advanced reasoning and synthesis.\n",
    "- **Self-Correction:** Programmatic generation of HTML reports.\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup Environment\n",
    "Prepare the workspace, install dependencies, and configure API keys."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Set working directory to project root (if running from notebooks/)\n",
    "import os\n",
    "import sys\n",
    "\n",
    "if os.path.basename(os.getcwd()) == \"notebooks\":\n",
    "    os.chdir(\"..\")\n",
    "    print(f\"📂 Changed working directory to: {os.getcwd()}\")\n",
    "\n",
    "# 2. Install Dependencies (if running on Colab/Kaggle)\n",
    "# !pip install -r requirements.txt -q\n",
    "\n",
    "from dotenv import load_dotenv\n",
    "load_dotenv()\n",
    "\n",
    "# 3. Verify API Key\n",
    "if not os.getenv(\"GEMINI_API_KEY\") and not os.getenv(\"GOOGLE_API_KEY\"):\n",
    "    print(\"❌ Error: API Key not found. Please check your .env file or Secrets.\")\n",
    "else:\n",
    "    print(\"✅ API Key loaded successfully.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Initialize Production Agents\n",
    "Import the real `LLMCoordinatorAgent` from our codebase. This agent orchestrates the DataLoader, Analyst, Recommender, and Critic."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import asyncio\n",
    "from agents.coordinator_llm import LLMCoordinatorAgent\n",
    "\n",
    "# Initialize the real Coordinator\n",
    "try:\n",
    "    coordinator = LLMCoordinatorAgent()\n",
    "    print(\"✅ Production Agents Initialized:\")\n",
    "    print(\"   - Coordinator\")\n",
    "    print(\"   - DataLoader (Tools: load_data)\")\n",
    "    print(\"   - Analyst (Tools: detect_anomalies, search_trends)\")\n",
    "    print(\"   - Recommender\")\n",
    "    print(\"   - Critic\")\n",
    "except ImportError as e:\n",
    "    print(f\"❌ Import Error: {e}. Make sure you are in the project root.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Execute Full Pipeline\n",
    "Run the analysis on `data/shipments_data.xlsx`. The system will:\n",
    "1. Load data metadata.\n",
    "2. Pass the **filepath** to the Analyst.\n",
    "3. Detect anomalies using Python tools.\n",
    "4. Generate strategic recommendations.\n",
    "5. Create an HTML report."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"🚀 Starting Autonomous Pipeline...\\n\")\n",
    "\n",
    "# Execute the pipeline (awaiting because Jupyter is async)\n",
    "# Note: We pass the config path, although defaults are usually handled internally\n",
    "final_summary = await coordinator.execute_pipeline(\"config/analysis_settings.json\")\n",
    "\n",
    "print(\"\\n✅ Pipeline Execution Complete.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Result Visualization\n",
    "Display the generated HTML report directly in the notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from IPython.display import HTML, display\n",
    "import os\n",
    "\n",
    "report_path = \"output/analysis_report.html\"\n",
    "\n",
    "if os.path.exists(report_path):\n",
    "    print(\"📊 Rendering Final Analysis Report:\")\n",
    "    with open(report_path, \"r\", encoding=\"utf-8\") as f:\n",
    "        html_content = f.read()\n",
    "    display(HTML(html_content))\n",
    "else:\n",
    "    print(\"⚠️ Report file not found. Please check logs above for errors.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
